In [ ]:
# open the image using stuff

import matplotlib.pyplot as plt
import numpy as np
import glob as glob
import torch
import cv2
import time

filename = '../coord_-5.4,+18.6_2_1X.tif'

image = np.memmap(filename, mode='c') # this should load the giant image into a np array in memory, 
#loading it in c mode prevents changes from affecting disk, only memory
print(image.shape)

In [ ]:
# preview some slices of the image
for i in range(0, image.shape[0], 100):
    img = cv2.cvtColor(image[i], cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(20,20))
    plt.imshow(img)
    plt.axis('off')
    plt.show()

In [ ]:
# run segment anything on some random slices of the image to test

def show_anns(anns):
    if len(anns) == 0:
        return
    sorted_anns = sorted(anns, key=(lambda x: x['area']), reverse=True)
    ax = plt.gca()
    ax.set_autoscale_on(False)

    img = np.ones((sorted_anns[0]['segmentation'].shape[0], sorted_anns[0]['segmentation'].shape[1], 4))
    img[:,:,3] = 0
    for ann in sorted_anns:
        m = ann['segmentation']
        color_mask = np.concatenate([np.random.random(3), [0.35]])
        img[m] = color_mask
    ax.imshow(img)

import sys
sys.path.append("..")

from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

sam_checkpoint = "../sam_vit_h_4b8939.pth"
model_type = "vit_h"

device = "cuda"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

mask_generator = SamAutomaticMaskGenerator(sam)
    
for i in range(0, image.shape[0], 100):
    img = cv2.cvtColor(image[i], cv2.COLOR_BGR2RGB)
    initial_time = time.time()
    masks = mask_generator.generate(img)
    print("Elapsed time: ", time.time() - initial_time)
    plt.figure(figsize=(20,20))
    plt.imshow(image)
    show_anns(masks)
    plt.axis('off')
    plt.show() 
    

In [ ]:
# run segment anything on all slices of the image